##### Это финальное задание по курсу «Обучение на размеченных данных».

В нем вы сравните логистическую регрессию и случайный лес на разных наборах признаков. В качестве данных будет использован Adult Data Set из репозитория UCI. В нем нужно предсказать, получает ли человек больше 50 000$ в год, или нет, по ряду признаков, таких как пол, образование, раса и др. Подробное описание можно найти по ссылке: https://archive.ics.uci.edu/ml/datasets/Adult.

In [1]:
import numpy as np
import pandas as pd

### Загрузка данных

В ходе задания будем использовать __train_data.csv__ для обучения моделей, на нем же и будем производить кросс-валидацию. В качестве отложенной выборки будем использовать __test_data.csv__

In [2]:
column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
                "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
                "hours-per-week", "native-country", "class"]

In [3]:
train = pd.read_csv("train_data.csv", sep=", ", header=None, engine="python", names=column_names)

In [4]:
test = pd.read_csv("test_data.csv", sep=", ", header=None, engine="python", names=column_names)

### 1. Пропущенные значения 

В обучающей выборке порядка 7% строк имеют пропущенные значения (вместо значения поля указан вопросительный знак __'?'__). В каких признаках в обучающей и тестовой выборке имеются пропущенные значения? Так как они все пропущены в категориальных признаках, то можно пока их ни на что не заменять, а просто считать еще одной категорией.

Переименуем колонки для удобства доступа:

In [5]:
X_train = train.rename(columns={'education-num': 'education_num', "marital-status": "marital_status", 
                                                                "capital-gain": "capital_gain" , "capital-loss": "capital_loss", 
                                                                "hours-per-week": "hours_per_week",
                                                                "native-country": "native_country"
                                                               })
X_test = test.rename(columns={'education-num': 'education_num', "marital-status": "marital_status", 
                                                                "capital-gain": "capital_gain" , "capital-loss": "capital_loss", 
                                                                "hours-per-week": "hours_per_week",
                                                                "native-country": "native_country"
                                                               })

In [6]:
X_train.shape == train.shape

True

In [7]:
y_train = X_train['class']
y_test = X_test['class']
y_train.head(8)

0    <=50K
1    <=50K
2    <=50K
3    <=50K
4    <=50K
5    <=50K
6    <=50K
7     >50K
Name: class, dtype: object

In [8]:
X_train = X_train.drop(['class'], axis=1)
X_test = X_test.drop(['class'], axis=1)

In [9]:
X_train[X_train == '?'].count()

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
dtype: int64

In [10]:
X_test[X_test == '?'].count()

age                 0
workclass         963
fnlwgt              0
education           0
education_num       0
marital_status      0
occupation        966
relationship        0
race                0
sex                 0
capital_gain        0
capital_loss        0
hours_per_week      0
native_country    274
dtype: int64

Видим, что значения пропущены в признаках с именами: "occupation", "workclass", "native_country". При кодировании будем считать "?" еще одной категорией.

### 2. Обучение на вещественных признаках

В этом разделе обучите модели только на вещественных признаках ("continuous" в описании данных). Обучите логистическую регрессию (linear_model.LogisticRegression) и случайный лес (ensemble.RandomForestClassifier) из sklearn. В первом случае подберите оптимальные параметры $penalty$ и $C$ на отрезке $[10^{-6}, 10^{6}]$ (по степеням $10$ с шагом $1$, начиная с $-6$), а во втором при фиксированном числе деревьев в 50 подберите $max\_depth$ и $min\_samples\_split$ из отрезка $[2, 14]$ с шагом в 2 и множества $\{1, 2, 4, 8\}$ соответственно. За целевую метрику качества возьмите AUC-ROC. В качестве схемы валидации используйте стратифицированную кросс-валидацию по 5-ти фолдам. Какие параметры оказались оптимальными?

Учтите, что целевая переменная в датасете является строкой. Поэтому для начала ее нужно перевести в бинарную величину. Также не забудьте отмасштабировать данные с помощью StandartScaler'а из модуля preprocessing.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score

In [12]:
# оставляем вещественные признаки
X_train_continuous = X_train.drop(['workclass', 'education', 'marital_status', 'occupation', 'relationship',
                       'race', 'sex', 'native_country'], axis=1)
X_train_continuous.columns

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [13]:
X_test_continuous = X_test.drop(['workclass', 'education', 'marital_status', 'occupation', 'relationship',
                       'race', 'sex', 'native_country'], axis=1)
X_test_continuous.columns

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [14]:
# масштабируем
scaler_train = StandardScaler()
X_train_continuous = scaler_train.fit_transform(X_train_continuous)

scaler_test = StandardScaler()
X_test_continuous = scaler_test.fit_transform(X_test_continuous)

In [15]:
# преобразуем ответы в два численных класса
y_train[y_train == '<=50K'] = 1
y_train[y_train == '>50K'] = 0
y_train = y_train.astype('int')

y_test[y_test == '<=50K'] = 1
y_test[y_test == '>50K'] = 0
y_test = y_test.astype('int')

In [16]:
# инициализируем модели
classifier_log = LogisticRegression(random_state=0, max_iter=10000)
classifier_randf = RandomForestClassifier(n_estimators=50, random_state=0)

# задаем параметры для сетки
parameters_grid_log = {
    'C': [10**i for i in range(-6, 7)],
}

parameters_grid_randf = {
    'max_depth': range(2, 15, 2),
    'min_samples_split': [1., 2, 4, 8],
}

grid_cv_log = GridSearchCV(classifier_log, parameters_grid_log, scoring = 'roc_auc', cv=5)
grid_cv_randf = GridSearchCV(classifier_randf, parameters_grid_randf, scoring = 'roc_auc', cv=5)


In [17]:
grid_cv_log.fit(X_train_continuous, y_train)
grid_cv_log.best_estimator_

LogisticRegression(C=0.1, max_iter=10000, random_state=0)

In [18]:
print(grid_cv_log.best_score_)
print(grid_cv_log.best_params_)

0.8316100306392272
{'C': 0.1}


In [19]:
grid_cv_randf.fit(X_train_continuous, y_train)
grid_cv_randf.best_estimator_

RandomForestClassifier(max_depth=12, n_estimators=50, random_state=0)

In [20]:
print(grid_cv_randf.best_score_)
print(grid_cv_randf.best_params_)

0.8634784804180331
{'max_depth': 12, 'min_samples_split': 2}


Посчитайте accuracy, precision, recall, f1-score и AUC-ROC на отложенной выборке для оптимальных алгоритмов. Какие они получились? Какой алгоритм лучше?

In [21]:
predicted_log = grid_cv_log.best_estimator_.predict(X_test_continuous)

In [22]:
predicted_randf = grid_cv_randf.best_estimator_.predict(X_test_continuous)

In [23]:
def print_score(predicted, target):
    print('AUC-ROC:', roc_auc_score(predicted, y_test))
    print('Accuracy', accuracy_score(y_test, predicted))
    print('F1-score:', f1_score(y_test, predicted))
    print('Recall:', recall_score(y_test, predicted))

In [24]:
print_score(predicted_log, y_test)

AUC-ROC: 0.7591389196647307
Accuracy 0.8127264910017812
F1-score: 0.8852853756725234
Recall: 0.9461198230800161


In [25]:
print_score(predicted_randf, y_test)

AUC-ROC: 0.8042857325334096
Accuracy 0.8347767336158712
F1-score: 0.8985441653466094
Recall: 0.9579412947326096


Как видим, лес лучше справился с задачей по всем метрикам

### 3. Категориальные признаки как есть

Теперь к вещественным добавьте категориальные признаки, заменив их на числа с помощью LabelEncoder из модуля preprocessing. Переподберите параметры для логистической регрессии и случайного леса аналогично прошлому пункту. Как изменилось качество моделей на тестовой выборке? Как вы можете это объяснить?

In [26]:
# т.к. LabelEncoder предназначен для одномерных массивов, воспользуемся OrdinalEncoder, хотя возможно и через цикл с помощью LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

X_train_cat = X_train.drop(["age", "fnlwgt", "education_num",
                "capital_gain", "capital_loss", "hours_per_week"], axis=1)
X_test_cat = X_test.drop(["age", "fnlwgt", "education_num",
                "capital_gain", "capital_loss", "hours_per_week"], axis=1)

encoder_train = OrdinalEncoder()
X_train_cat_enc = encoder_train.fit_transform(X_train_cat)

encoder_test = OrdinalEncoder()
X_test_cat_enc = encoder_test.fit_transform(X_test_cat)

X_train = np.hstack((X_train_cat_enc, X_train_continuous))
X_test = np.hstack((X_test_cat_enc, X_test_continuous))

In [27]:
X_train.shape[0] == train.shape[0]

True

In [28]:
X_test.shape[0] == test.shape[0]

True

In [29]:
grid_cv_log_cat_enc = GridSearchCV(classifier_log, parameters_grid_log, scoring='roc_auc', cv=5)
grid_cv_randf_cat_enc = GridSearchCV(classifier_randf, parameters_grid_randf, scoring='roc_auc', cv=5)

In [30]:
grid_cv_log_cat_enc.fit(X_train, y_train)
grid_cv_log_cat_enc.best_estimator_

LogisticRegression(C=0.1, max_iter=10000, random_state=0)

In [31]:
grid_cv_randf_cat_enc.fit(X_train, y_train)
grid_cv_randf_cat_enc.best_estimator_

RandomForestClassifier(max_depth=12, n_estimators=50, random_state=0)

In [32]:
print(grid_cv_log_cat_enc.best_score_)
print(grid_cv_log_cat_enc.best_params_)

0.8538968851535212
{'C': 0.1}


In [33]:
grid_cv_randf_cat_enc.fit(X_train, y_train)
grid_cv_randf_cat_enc.best_estimator_

RandomForestClassifier(max_depth=12, n_estimators=50, random_state=0)

In [34]:
print(grid_cv_randf_cat_enc.best_score_)
print(grid_cv_randf_cat_enc.best_params_)

0.9166620837732443
{'max_depth': 12, 'min_samples_split': 2}


In [35]:
predicted_log_cat_enc = grid_cv_log_cat_enc.best_estimator_.predict(X_test)
predicted_randf_cat_enc = grid_cv_randf_cat_enc.best_estimator_.predict(X_test)

In [36]:
print_score(predicted_log_cat_enc, y_test)

AUC-ROC: 0.774595203055598
Accuracy 0.8245193784165592
F1-score: 0.8913068289899182
Recall: 0.9420184961801367


In [37]:
print_score(predicted_randf_cat_enc, y_test)

AUC-ROC: 0.8295704491950132
Accuracy 0.8586081935999017
F1-score: 0.9115227919132909
Recall: 0.9535987133092079


### 4. Бинарное кодирование категориальных признаков

А теперь замените категориальные признаки из прошлого пункта на бинарно закодированные. Опять переподберите параметры для моделей и проверьте качество на тестовой выборке. Как изменилось качество относительно предыдущего пункта? Как вы можете это объяснить?

In [39]:
from sklearn.preprocessing import LabelBinarizer

binarizer_train = LabelBinarizer()
for col in X_train_cat.columns:
    X_train_cat[col] = binarizer_train.fit_transform(X_train_cat[col])

binarizer_test = LabelBinarizer()
for col in X_test_cat.columns:
    X_test_cat[col] = binarizer_test.fit_transform(X_test_cat[col])

X_train2 = np.hstack((X_train_cat, X_train_continuous))
X_test2 = np.hstack((X_test_cat, X_test_continuous))

In [43]:
grid_cv_log_cat_binarize = GridSearchCV(classifier_log, parameters_grid_log, scoring = 'roc_auc', cv=5)
grid_cv_randf_cat_binarize = GridSearchCV(classifier_randf, parameters_grid_randf, scoring = 'roc_auc', cv=5)

In [45]:
grid_cv_log_cat_binarize.fit(X_train2, y_train)
grid_cv_log_cat_binarize.best_estimator_

LogisticRegression(C=0.1, max_iter=10000, random_state=0)

In [46]:
print(grid_cv_log_cat_binarize.best_score_)
print(grid_cv_log_cat_binarize.best_params_)

0.8769851259244339
{'C': 0.1}


In [47]:
grid_cv_randf_cat_binarize.fit(X_train2, y_train)
grid_cv_randf_cat_binarize.best_estimator_

RandomForestClassifier(max_depth=14, min_samples_split=4, n_estimators=50,
                       random_state=0)

In [48]:
print(grid_cv_randf_cat_binarize.best_score_)
print(grid_cv_randf_cat_binarize.best_params_)

0.8993103325921382
{'max_depth': 14, 'min_samples_split': 4}


In [50]:
predicted_log_cat_binarize = grid_cv_log_cat_binarize.best_estimator_.predict(X_test2)
predicted_randf_cat_binarize = grid_cv_randf_cat_binarize.best_estimator_.predict(X_test2)

## Итоги

Log Regression:

In [53]:
print(' First model', '\nAUC-ROC 1:', roc_auc_score(predicted_log, y_test))
print('Accuracy 1', accuracy_score(y_test, predicted_log))
print('F1-score 1:', f1_score(y_test, predicted_log))
print('Recall 1:', recall_score(y_test, predicted_log))

print(' Second model', '\nAUC-ROC 3:', roc_auc_score(predicted_log_cat_enc, y_test))
print('Accuracy 3', accuracy_score(y_test, predicted_log_cat_enc))
print('F1-score 3:', f1_score(y_test, predicted_log_cat_enc))
print('Recall3:', recall_score(y_test, predicted_log_cat_enc))

print(' Third model', '\nAUC-ROC 4:', roc_auc_score(predicted_log_cat_binarize, y_test))
print('Accuracy 4:', accuracy_score(y_test, predicted_log_cat_binarize))
print('F1-score 4:', f1_score(y_test, predicted_log_cat_binarize))
print('Recall 4:', recall_score(y_test, predicted_log_cat_binarize))

 First model 
AUC-ROC 1: 0.7591389196647307
Accuracy 1 0.8127264910017812
F1-score 1: 0.8852853756725234
Recall 1: 0.9461198230800161
 Second model 
AUC-ROC 3: 0.774595203055598
Accuracy 3 0.8245193784165592
F1-score 3: 0.8913068289899182
Recall3: 0.9420184961801367
 Third model 
AUC-ROC 4: 0.7893440624965217
Accuracy 4: 0.8376021128923284
F1-score 4: 0.898048893344644
Recall 4: 0.9364696421391234


Random Forests:

In [55]:
print(' First model', '\nAUC-ROC 1:', roc_auc_score(predicted_randf, y_test))
print('Accuracy 1:', accuracy_score(y_test, predicted_randf))
print('F1-score 1:', f1_score(y_test, predicted_randf))
print('Recall 1:', recall_score(y_test, predicted_randf))

print(' Second model', '\nAUC-RO 3:', roc_auc_score(predicted_randf_cat_enc, y_test))
print('Accuracy 3:', accuracy_score(y_test, predicted_randf_cat_enc))
print('F1-score 3:', f1_score(y_test, predicted_randf_cat_enc))
print('Recall 3:', recall_score(y_test, predicted_randf_cat_enc))

print(' Third model', '\nAUC-ROC 4:', roc_auc_score(predicted_randf_cat_binarize, y_test))
print('Accuracy 4:', accuracy_score(y_test, predicted_randf_cat_binarize))
print('F1-score 4:', f1_score(y_test, predicted_randf_cat_binarize))
print('Recall 4:', recall_score(y_test, predicted_randf_cat_binarize))

 First model 
AUC-ROC 1: 0.8042857325334096
Accuracy 1: 0.8347767336158712
F1-score 1: 0.8985441653466094
Recall 1: 0.9579412947326096
 Second model 
AUC-RO 3: 0.8295704491950132
Accuracy 3: 0.8586081935999017
F1-score 3: 0.9115227919132909
Recall 3: 0.9535987133092079
 Third model 
AUC-ROC 4: 0.8258405290462604
Accuracy 4: 0.8515447454087587
F1-score 4: 0.9078992493236292
Recall 4: 0.958021712907117
